# 🌀 Quantum Tetris 🎮✨

### Install the required dependencies for Qiskit

In [ ]:
! pip install qiskit

### 1: Quantum Backend and Communication Bridge


---


This code connects a Python backend to the Quokka quantum simulator to generate true quantum randomness and control game speed. It includes:

*   QRNG (qrng_0_to_63): Builds a 6-qubit Hadamard circuit, sends it to Quokka, and returns a random integer (0–63).
*   Grover-Based Speed: Uses Grover’s algorithm to pick a target state and adjust game speed based on whether the measured result matches the target.

*   QPE Speed Control: Applies Quantum Phase Estimation for finer speed control, mapping phase estimates to speed factors.

*   Comm Channel: A Jupyter Comm ("quantum") receives messages from JavaScript to either:

  *   Select a game piece based on quantum randomness.
  *   Return a quantum-derived speed factor using Grover or QPE.

This bridges quantum computing with browser-based gameplay via real-time messaging.

In [ ]:
import json, requests, warnings, qiskit
from qiskit import QuantumCircuit, transpile
from qiskit.qasm3 import dumps
from ipykernel.comm import Comm
import numpy as np
from math import pi

warnings.filterwarnings("ignore")  # hide self-signed HTTPS warnings

# 1 ▸ Helper — push OpenQASM to Quokka and get raw counts back
def send_to_quokka(program: str, count: int = 1, host: str = "quokka3"):
    """
    Args
    ----
    program : OpenQASM 3 code as a plain string.
    count   : Int — number of shots (we only need 1 shot per collapse).
    host    : Quokka sub-domain (e.g. 'quokka3').

    Returns
    -------
    List of lists — one inner list per shot, each element 0/1.
    """
    url = f"http://{host}.quokkacomputing.com/qsim/qasm"
    resp = requests.post(url,
                         json={"script": program, "count": count},
                         verify=False, timeout=10)
    resp.raise_for_status()
    return json.loads(resp.content)["result"]["c"]

# 2 ▸ QRNG — one 6-qubit Hadamard ladder (0–63 uniform)
def qrng_0_to_63() -> int:
    qc = QuantumCircuit(6, 6)
    qc.h(range(6))
    qc.measure(range(6), range(6))
    bits = send_to_quokka(dumps(qc))[0]        # first (only) shot
    return int("".join(map(str, bits)), 2)     # binary → decimal

# 3 ▸ Grover-based Speed Feature
def grover_speed_oracle(n_qubits=3, target_states=None):
    """
    Create a quantum circuit that implements a Grover oracle for speed control.

    Args:
        n_qubits: Number of qubits
        target_states: List of target states to mark. If None, randomly choose states.

    Returns:
        oracle_circuit: Quantum circuit implementing the oracle
        target_states: The marked states
    """
    if target_states is None:
        # Randomly choose 1 or 2 states to mark
        num_targets = 1 if qrng_0_to_63() % 2 == 0 else 2
        possible_states = list(range(2**n_qubits))
        # Use QRNG to select target states
        target_states = []
        for _ in range(num_targets):
            if not possible_states:
                break
            idx = qrng_0_to_63() % len(possible_states)
            target_states.append(possible_states.pop(idx))

    # Create oracle circuit
    oracle = QuantumCircuit(n_qubits)

    for target in target_states:
        # Convert target to binary and pad with zeros
        target_bin = format(target, f'0{n_qubits}b')

        # Apply X gates to qubits that should be 0
        for i, bit in enumerate(target_bin):
            if bit == '0':
                oracle.x(i)

        # Apply multi-controlled Z gate (via H, multi-controlled X, H)
        oracle.h(n_qubits-1)
        oracle.mcx(list(range(n_qubits-1)), n_qubits-1)
        oracle.h(n_qubits-1)

        # Undo X gates
        for i, bit in enumerate(target_bin):
            if bit == '0':
                oracle.x(i)

    return oracle, target_states

def grover_diffuser(n_qubits):
    """Create the diffusion operator for Grover's algorithm"""
    qc = QuantumCircuit(n_qubits)

    # Apply H gates to all qubits
    for qubit in range(n_qubits):
        qc.h(qubit)

    # Apply X gates to all qubits
    for qubit in range(n_qubits):
        qc.x(qubit)

    # Apply multi-controlled Z gate
    qc.h(n_qubits-1)
    qc.mcx(list(range(n_qubits-1)), n_qubits-1)
    qc.h(n_qubits-1)

    # Undo X gates
    for qubit in range(n_qubits):
        qc.x(qubit)

    # Undo H gates
    for qubit in range(n_qubits):
        qc.h(qubit)

    return qc

def run_grover_speed_algorithm(n_qubits=3, iterations=1):
    """
    Run Grover's algorithm for speed control.

    Args:
        n_qubits: Number of qubits
        iterations: Number of Grover iterations

    Returns:
        measured_state: The measured state
        speed_factor: A speed factor derived from the state
        target_states: The marked states
    """
    # Create oracle
    oracle, target_states = grover_speed_oracle(n_qubits)

    # Create main circuit
    qc = QuantumCircuit(n_qubits, n_qubits)

    # Initialize with Hadamard
    for qubit in range(n_qubits):
        qc.h(qubit)

    # Repeat Grover iterations
    for _ in range(iterations):
        # Apply oracle
        qc = qc.compose(oracle)

        # Apply diffuser
        qc = qc.compose(grover_diffuser(n_qubits))

    # Measure all qubits
    qc.measure(range(n_qubits), range(n_qubits))

    # Execute the circuit
    bits = send_to_quokka(dumps(qc))[0]
    measured_state = int("".join(map(str, bits)), 2)

    # Calculate a speed factor between 0.5 and 2.0
    # If we hit a target state, speed up (factor > 1)
    # If we miss, slow down (factor < 1)
    if measured_state in target_states:
        # We hit a target state! Speed up
        speed_factor = 1.5 + (measured_state / (2**n_qubits)) * 0.5  # Between 1.5 and 2.0
    else:
        # We missed a target state. Slow down
        speed_factor = 0.5 + (measured_state / (2**n_qubits)) * 0.5  # Between 0.5 and 1.0

    return measured_state, speed_factor, target_states

# 4 ▸ Quantum Phase Estimation for precise speed control
def qpe_speed_control(precision=3):
    """
    Use Quantum Phase Estimation to get a more precise speed control value.

    Args:
        precision: Number of qubits for precision (3-5 recommended)

    Returns:
        phase: Estimated phase (float between 0 and 1)
        speed_factor: Speed factor derived from the phase
    """
    # Target qubit
    t_qubits = 1
    # Precision qubits
    p_qubits = precision

    # Create circuit
    qc = QuantumCircuit(p_qubits + t_qubits, p_qubits)

    # Initialize target qubit in |1⟩ state
    qc.x(p_qubits)

    # Apply H gates to all precision qubits
    for qubit in range(p_qubits):
        qc.h(qubit)

    # Apply controlled unitary operations
    # Using controlled-U^(2^j) operations
    for j in range(p_qubits):
        # Create random rotation angle for demonstration
        rotation_angle = (qrng_0_to_63() / 64.0) * 2 * pi
        # Apply controlled rotation
        for _ in range(2**j):
            qc.cp(rotation_angle, j, p_qubits)

    # Apply inverse QFT to precision qubits
    # Simple implementation (not optimized)
    for j in range(p_qubits):
        qc.h(j)
        for k in range(j):
            qc.cp(-pi/(2**(j-k)), k, j)

    # Measure precision qubits
    qc.measure(range(p_qubits), range(p_qubits))

    # Execute the circuit
    bits = send_to_quokka(dumps(qc))[0]
    measured_value = int("".join(map(str, bits)), 2)

    # Convert to phase between 0 and 1
    phase = measured_value / (2**p_qubits)

    # Map to speed factor (between 0.25 and 2.0)
    speed_factor = 0.25 + phase * 1.75

    return phase, speed_factor

# 5 ▸ Comm bridge — talk to JavaScript
comm = Comm(target_name="quantum")

@comm.on_msg
def _handle_js(msg):
    """
    Expected JS payload:
        {'probs': [p0, p1, …, pn]} for piece selection
        OR {'speed_request': true} for speed feature

    We send back:
        {'choice': i} for piece selection
        OR {'speed_factor': f} for speed feature
    """
    data = msg["content"]["data"]

    if 'probs' in data:
        # Normal piece selection logic
        probs = data["probs"]
        u = qrng_0_to_63() / 64                # → float in [0,1)
        acc = 0
        for i, p in enumerate(probs):
            acc += p
            if u <= acc:
                comm.send({"choice": i})
                return
        # extremely unlikely float-precision fall-back
        comm.send({"choice": len(probs) - 1})

    elif 'speed_request' in data:
        # Speed feature requested
        if data.get('use_grover', True):
            # Use Grover's algorithm
            n_qubits = data.get('n_qubits', 3)
            iterations = data.get('iterations', 1)
            state, speed_factor, targets = run_grover_speed_algorithm(n_qubits, iterations)
            comm.send({
                "speed_factor": speed_factor,
                "state": state,
                "targets": targets,
                "iteration_count": iterations
            })
        else:
            # Use QPE for more precise control
            precision = data.get('precision', 3)
            phase, speed_factor = qpe_speed_control(precision)
            comm.send({
                "speed_factor": speed_factor,
                "phase": phase,
                "precision": precision
            })

print("✅ Quantum speed control ready")

✅ Quantum speed control ready


### 2: Tetris Front-End & Quantum Collapse


---


This section runs the Tetris game in JavaScript using HTML5 canvas. It includes drawing, movement, collision, scoring, and WASD / Arrow key controls. When a piece hits the 'measurement line', the game calls 'collapseInPython', sending the 'probs' array to Python. Python returns an index based on those probabilities, and JS uses it to finalise the piece’s shape, introducing quantum randomness into gameplay.


---


#### Communication
The Jupyter Comm channel connects JS and Python. JS sends probs; Python runs qrng_0_to_63, picks a result, and sends it back. That result “collapses” the quantum piece into a specific shape, letting quantum mechanics influence game behavior.

In [ ]:
from IPython.display import HTML, display
display(HTML(r"""

<div style="display:flex;gap:16px;align-items:flex-start">
  <div>
    <canvas id="tetris"></canvas>

    <!-- Speed control UI -->
    <div id="speed-control" style="margin-top:10px;font-family:monospace;color:#eee;display:flex;align-items:center;gap:10px;">
      <button id="speed-btn" style="background:#3877FF;color:white;border:none;padding:8px 12px;cursor:pointer;font-family:monospace;border-radius:4px;">
        QUANTUM SPEED
      </button>
      <div id="speed-indicator" style="width:120px;height:12px;background:#333;position:relative;">
        <div id="speed-bar" style="position:absolute;top:0;left:0;height:100%;width:50%;background:#72FF0D;transition:width 0.3s,background 0.3s;"></div>
        <div id="normal-marker" style="position:absolute;top:0;left:50%;width:2px;height:100%;background:white;"></div>
      </div>
      <span id="speed-value">1.0x</span>
    </div>
  </div>

  <div style="color:#eee;font:16px monospace">
    <h3 style="margin:4px 0 8px">SCORE</h3>
    <div id="score">0</div>

    <!-- Game status display -->
    <div id="game-status" style="margin:8px 0;font-weight:bold;color:#FF8E0D;">PRESS 'O' TO START</div>

    <!-- Controls legend -->
    <div style="margin-top:12px;line-height:1.4">
      <b>CONTROLS</b><br>
      A/←&nbsp;·&nbsp;Left &nbsp;&nbsp;<br>
      D/→&nbsp;·&nbsp;Right<br>
      S/↓&nbsp;·&nbsp;Soft&nbsp;drop<br>
      W/↑&nbsp;·&nbsp;Rotate<br>
      Space&nbsp;·&nbsp;Hard&nbsp;drop<br>
      P&nbsp;·&nbsp;Pause<br>
      Q&nbsp;·&nbsp;Quantum&nbsp;Speed<br>
      O&nbsp;·&nbsp;Start&nbsp;Game
    </div>

    <!-- Quantum state display -->
    <div style="margin-top:20px;line-height:1.4;max-width:200px;">
      <b>QUANTUM STATE</b><br>
      <div id="q-state">Waiting...</div>
    </div>
  </div>
</div>

<script>
let collapseInPython;
const hasKernel = (typeof Jupyter!=='undefined' &&
                   Jupyter.notebook &&
                   Jupyter.notebook.kernel);

if (hasKernel) {
  const qcomm = Jupyter.notebook.kernel.comm_manager.new_comm("quantum");
  collapseInPython = probs => new Promise(res=>{
    qcomm.on_msg = m => res(m.content.data.choice);
    qcomm.send({probs});
  });

  // Add speed control function
  requestQuantumSpeed = () => new Promise(res => {
    qcomm.on_msg = m => res(m.content.data);
    qcomm.send({speed_request: true, use_grover: true, n_qubits: 3, iterations: 1});
  });

  console.log("Comm channel ready → Quokka randomness enabled");
} else {
  collapseInPython = probs => new Promise(res=>{
    const u=Math.random(); let acc=0;
    probs.forEach((p,i)=>{acc+=p;if(u<=acc)res(i);});
  });

  // Fallback speed control function
  requestQuantumSpeed = () => new Promise(res => {
    const speedFactor = 0.5 + Math.random() * 1.5; // between 0.5 and 2.0
    res({speed_factor: speedFactor, state: "fallback", targets: []});
  });

  console.warn("No kernel found → fallback RNG in use");
}

// 1 ▸  CONSTANTS & BOARD ----------------------------------------------
const S=28, W=12, H=22, MEASURE_Y=Math.floor(H/3);
const cvs=document.getElementById("tetris");
cvs.width=W*S; cvs.height=H*S;

const ctx=cvs.getContext("2d");
ctx.scale(S,S); ctx.lineWidth=0.07;

const arena=Array.from({length:H},_=>Array(W).fill(0));
const COLORS=["#000","#FF0D72","#0DC2FF","#FF8E0D",
              "#FFE138","#3877FF","#72FF0D","#B207FF"];
const SHAPES={T:[[0,1,0],[1,1,1]],O:[[2,2],[2,2]],
              L:[[0,0,3],[3,3,3]],J:[[4,0,0],[4,4,4]],
              I:[[5,5,5,5]],S:[[0,6,6],[6,6,0]],
              Z:[[7,7,0],[0,7,7]]};
function superpositionPool(){
  const ids=Object.keys(SHAPES), p=1/ids.length;
  return ids.map(id=>({id,matrix:SHAPES[id].map(r=>r.slice()),prob:p}));
}

// 2 ▸  STATE -----------------------------------------------------------
const player={pos:{x:0,y:0},superpos:[],matrix:null,
              collapsed:false,flickerIdx:0};
let score=0, $score=document.getElementById("score");
const addScore=n=>{$score.textContent=(score+=n*100);};

// Game state
let gameStarted = false;
let gamePaused = false;
let gameOver = false;
const $gameStatus = document.getElementById("game-status");

// Speed control state
let speedFactor = 1.0;
let normalDropInt = 500;
let dropInt = normalDropInt;
let speedEffectDuration = 10000; // 10 seconds
let speedEffectActive = false;
let speedTimer = null;
let autoSpeedChange = true; // Flag to control auto speed changes
const $speedBar = document.getElementById("speed-bar");
const $speedValue = document.getElementById("speed-value");
const $qState = document.getElementById("q-state");

// Flag to track if a piece has just passed the measurement line
let justPassedMeasureLine = false;

function reset(){
  player.superpos=superpositionPool();
  player.matrix=null; player.collapsed=false; player.flickerIdx=0;

  // Get the maximum width of any piece in the superposition
  const maxPieceWidth = Math.max(...player.superpos.map(p => {
    return p.matrix[0].length;
  }));

  // Adjust spawn position to center the piece and ensure it won't clip the right side
  player.pos.y = 0;
  player.pos.x = Math.floor((W - maxPieceWidth) / 2);

  // If still colliding after reset, check if we've reached MEASURE_Y (game over condition)
  if(collide()) {
    // Check if we're above or at the measurement line
    if (player.pos.y <= MEASURE_Y) {
      arena.forEach(r=>r.fill(0));
      score=0;
      addScore(0);

      if (gameStarted && !gameOver) {
        endGame();
      }
    } else {
      // If below measurement line, try again with a higher position
      player.pos.y--;
      if(collide()) {
        // If still colliding, clear the board and end the game
        arena.forEach(r=>r.fill(0));
        score=0;
        addScore(0);

        if (gameStarted && !gameOver) {
          endGame();
        }
      }
    }
  }
}

function startGame() {
  if (gameOver || !gameStarted) {
    gameStarted = true;
    gameOver = false;
    arena.forEach(r=>r.fill(0));
    score = 0;
    addScore(0);
    $gameStatus.textContent = "GAME ON";
    $gameStatus.style.color = "#72FF0D";
    reset();
  }
}

function endGame() {
  gameOver = true;
  gameStarted = false;
  $gameStatus.textContent = "GAME OVER - PRESS 'O' TO RESTART";
  $gameStatus.style.color = "#FF0D72";
}

function pauseGame() {
  gamePaused = !gamePaused;
  $gameStatus.textContent = gamePaused ? "PAUSED" : "GAME ON";
  $gameStatus.style.color = gamePaused ? "#FFE138" : "#72FF0D";
}

const currentMatrix=()=>player.collapsed?player.matrix
                                       :player.superpos[player.flickerIdx].matrix;

// 2.1 ▸ collision / merge / sweep --------------------------
function collide(mat=currentMatrix()){const o=player.pos;
  for(let y=0;y<mat.length;++y)
    for(let x=0;x<mat[y].length;++x)
      if(mat[y][x]&&(arena[y+o.y]&&arena[y+o.y][x+o.x])!==0)return true;
  return false;}

function merge(){currentMatrix().forEach((row,y)=>row.forEach((v,x)=>{
  if(v)arena[y+player.pos.y][x+player.pos.x]=v;}));sweep();}

function sweep(){
  let cleared=0;
  outer:for(let y=H-1;y>=0;--y){
    for(let x=0;x<W;++x)if(!arena[y][x])continue outer;
    arena.splice(y,1);arena.unshift(Array(W).fill(0));++y;++cleared;
  }
  if(cleared)addScore(cleared);

  // Check if any piece has reached the top of the bottom box (MEASURE_Y)
  checkTopReached();
}

// Check if any piece has reached the measurement line or above
function checkTopReached() {
  // Check if any blocks are at or above the measurement line
  for (let y = 0; y <= MEASURE_Y; y++) {
    for (let x = 0; x < W; x++) {
      if (arena[y][x] !== 0) {
        endGame();
        return;
      }
    }
  }
}

// 3 ▸  QUANTUM COLLAPSE -----------------------------------------------
async function collapse(){
  const idx=await collapseInPython(player.superpos.map(p=>p.prob));
  player.matrix=player.superpos[idx].matrix;
  player.collapsed=true;

  // Mark that we just passed the measure line
  justPassedMeasureLine = true;
}

// Speed control functions
async function activateQuantumSpeed() {
  if (!gameStarted || gameOver) return;

  const result = await requestQuantumSpeed();
  applySpeedChange(result);
}

async function applySpeedChange(result) {
  speedFactor = result.speed_factor;
  dropInt = normalDropInt / speedFactor;

  // Update UI
  $speedBar.style.width = `${Math.min(speedFactor * 50, 100)}%`;
  $speedBar.style.background = speedFactor > 1 ? '#FF8E0D' : '#0DC2FF';
  $speedValue.textContent = `${speedFactor.toFixed(2)}x`;

  // Update quantum state display
  if (result.state !== "fallback") {
    // Format measured state and targets for display
    const stateInfo = `Measured: ${result.state}<br>` +
                      `Targets: ${result.targets.join(', ')}<br>` +
                      `Speed: ${speedFactor.toFixed(2)}x`;
    $qState.innerHTML = stateInfo;
  } else {
    $qState.innerHTML = "Using fallback RNG";
  }
}

// 4 ▸  GAME LOOP -------------------------------------------------------
let dropCtr=0,last=0,flickCtr=0,flickInt=100;
function update(t=0){
  if(gameOver || !gameStarted || gamePaused){
    requestAnimationFrame(update);
    return;
  }

  const dt=t-last; last=t; dropCtr+=dt; flickCtr+=dt;
  if(flickCtr>flickInt&&!player.collapsed){
    player.flickerIdx=(player.flickerIdx+1)%player.superpos.length; flickCtr=0;}
  if(dropCtr>dropInt){playerDrop(); dropCtr=0;}
  draw(); requestAnimationFrame(update);
}

async function playerDrop(){
  if (!gameStarted || gameOver) return;

  const oldY = player.pos.y;
  player.pos.y++;

  // Check if we're passing the measure line
  if(!player.collapsed && oldY < MEASURE_Y && player.pos.y >= MEASURE_Y) {
    await collapse();

    // Apply automatic speed change after measurement line
    if (autoSpeedChange && justPassedMeasureLine) {
      activateQuantumSpeed();
      justPassedMeasureLine = false;
    }
  }

  if(collide()){
    player.pos.y--;
    if(!player.collapsed) await collapse();
    merge(); reset();
  }
}

// 4.1 ▸  RENDER ---------------------------------------------------------
function drawMatrix(mat,off){mat.forEach((row,y)=>row.forEach((v,x)=>{
  if(v){ctx.fillStyle=COLORS[v];
    ctx.fillRect(x+off.x,y+off.y,1,1);
    ctx.strokeStyle="#111";ctx.strokeRect(x+off.x,y+off.y,1,1);}}));}
function draw(){
  ctx.fillStyle="#000";ctx.fillRect(0,0,W,H);

  // Draw measurement line
  ctx.fillStyle="rgba(255,255,255,0.18)";
  ctx.fillRect(0,MEASURE_Y,W,0.15);

  // Draw game board and current piece
  drawMatrix(arena,{x:0,y:0});

  // Only draw current piece if game is active
  if (gameStarted && !gameOver) {
    drawMatrix(currentMatrix(),player.pos);
  }

  // Draw "Game Over" or "Press O to Start" overlay if needed
  if (gameOver || !gameStarted) {
    // Semi-transparent overlay
    ctx.fillStyle = "rgba(0,0,0,0.5)";
    ctx.fillRect(0, 0, W, H);
  }
}

// 5 ▸  CONTROLS  (ASWD + Arrow Keys + Space + P + Q + O) -----------------
document.addEventListener("keydown",async e=>{
  const k=e.key.toLowerCase();

  // Start game with 'o' key
  if(k==='o'){
    startGame();
    e.preventDefault();
    return;
  }

  // Only process other controls if game is active
  if(!gameStarted || gameOver) return;

  if("aswdpq ".includes(k) ||
     ["arrowleft", "arrowright", "arrowup", "arrowdown"].includes(k)) {
    e.preventDefault(); // stop page scroll
  }

  if(k==='p'){
    pauseGame();
    return;
  }

  if(gamePaused) return;

  if(k==='a' || k==='arrowleft'){
    player.pos.x--;
    if(collide()) player.pos.x++;
  }
  else if(k==='d' || k==='arrowright'){
    player.pos.x++;
    if(collide()) player.pos.x--;
  }
  else if(k==='s' || k==='arrowdown'){
    playerDrop();
  }
  else if(k==='w' || k==='arrowup'){
    rotate(1);
  }
  else if(k==='q'){
    activateQuantumSpeed();
  }
  else if(k===' '){
    await hardDrop();
  }
});

// Add click handler for speed button
document.getElementById('speed-btn').addEventListener('click', () => {
  if (gameStarted && !gameOver && !gamePaused) {
    activateQuantumSpeed();
  }
});

async function hardDrop(){
  if (!gameStarted || gameOver || gamePaused) return;

  // Store the position before the drop to check if we pass the measure line
  const oldY = player.pos.y;
  let passedMeasureLine = false;

  while(!collide()){
    player.pos.y++;

    // Check if we're passing the measure line during this hard drop
    if(!player.collapsed && oldY < MEASURE_Y && player.pos.y >= MEASURE_Y) {
      await collapse();
      passedMeasureLine = true;
    }
  }

  player.pos.y--;
  if(!player.collapsed) await collapse();

  // Apply automatic speed change if we passed the measurement line
  if (autoSpeedChange && passedMeasureLine) {
    activateQuantumSpeed();
  }

  merge();
  reset();
}

function rotate(dir){
  if(!player.collapsed || !gameStarted || gameOver || gamePaused) return;
  const m = player.matrix;

  // Create a rotated copy of the matrix
  const rotated = m[0].map((_, colIndex) =>
    m.map(row => row[colIndex]).reverse()
  );

  // Save the original matrix and position in case we need to revert
  const originalMatrix = m;
  const originalX = player.pos.x;

  // Try the rotation
  player.matrix = rotated;

  // If the rotation causes collision, try to adjust the position
  if (collide()) {
    // First try moving left or right to accommodate
    const pieceWidth = rotated[0].length;
    let offset = 1;
    let success = false;

    // Try moving left/right up to the width of the piece
    while (offset <= pieceWidth) {
      // Try right
      player.pos.x += offset;
      if (!collide()) {
        success = true;
        break;
      }

      // Try left
      player.pos.x = originalX - offset;
      if (!collide()) {
        success = true;
        break;
      }

      // Reset and increase offset
      player.pos.x = originalX;
      offset++;
    }

    // If all adjustments fail, revert to the original state
    if (!success) {
      player.matrix = originalMatrix;
      player.pos.x = originalX;
    }
  }
}

reset(); update();
</script>
"""))